In [6]:
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
import os
import autogen
from autogen.coding import LocalCommandLineCodeExecutor
from dotenv import load_dotenv


# 1. 환경 설정 및 라이브러리 로드
load_dotenv()  # .env에서 API 키 로드

filter_dict = {"tags": "gpt-4o-mini"}
llm_config = autogen.LLMConfig(
    api_type="openai",
    model="gpt-4o-mini",
    api_key=os.environ.get("OPENAI_API_KEY"),
    cache_seed=41,
    temperature=0
)

In [ ]:
user_proxy = autogen.UserProxyAgent(
        name="user_proxy",
        human_input_mode="NEVER",  # 자동 응답, 수동 입력 비활성화
        default_auto_reply="Please analyze the stock data, financial statements, and news.",
        is_termination_msg=lambda msg: False,  
        code_execution_config={
            "executor": LocalCommandLineCodeExecutor(work_dir="groupchat_tutorial"),
        }
    )

stock_analyzer = autogen.AssistantAgent(
    name="stock_analyzer",
    llm_config = llm_config,
    system_message="You are a stock price analysis expert. Analyze historical price data, trends, and provide actionable stock insights."
)

financial_analyzer = autogen.AssistantAgent(
    name="financial_analyzer",
    llm_config = llm_config,
    system_message="""You are a financial statement analysis expert." \
    "  Make a code that can gather financial statements related to the company of interest."""
)

news_analyzer = autogen.AssistantAgent(
    name="news_analyzer",
    llm_config = llm_config,
    system_message="""You are a news analysis expert. 
        Make a code that can gather a news related to the company of interest."""
)

# 4. 총괄 전략 에이전트
aggregator = autogen.AssistantAgent(
    name="aggregator",
    llm_config = llm_config,
    system_message=(
        "You are an investment strategist. Use WebSearchTool to gather up-to-date market information, "
        "integrate analyses from the stock, financial, and news agents, then write Python code that computes an investment decision. "
        "Finally, recommend whether to invest or not, and explain your reasoning."
        "After your final recommendation please say \"TERMINATE\""
    ), 
    is_termination_msg=lambda msg: (
            'TSLA' in msg and any(keyword in msg for keyword in ['TERMINATE','Buy', 'Hold', 'Sell'])
        )
)


In [8]:
team = autogen.GroupChat(
    agents=[user_proxy, stock_analyzer, financial_analyzer, news_analyzer, aggregator],
    messages=[],
    max_round=50,
)
manager = autogen.GroupChatManager(groupchat=team, llm_config=llm_config)

In [9]:
chat_result = user_proxy.initiate_chat(
    manager, 
    message="""Hello investment_team: 
    please fetch and analyze the last five years of TSLA closing prices via stock_price_tool, 
    evaluate their latest and trailing three quarters of financial statements, 
    gather and summarize the past three months of relevant market news and sentiment via WebSearchTool,
    then write and execute Python code to simulate 1-year and 3-year expected returns under conservative, base, and aggressive scenarios, 
    and finally present each ticker’s Buy/Hold/Sell recommendation, portfolio weight suggestion, and the rationale—including key quantitative metrics and qualitative drivers—
    in a single cohesive Markdown report.
    """
)

user_proxy (to chat_manager):

Hello investment_team: 
    please fetch and analyze the last five years of TSLA closing prices via stock_price_tool, 
    evaluate their latest and trailing three quarters of financial statements, 
    gather and summarize the past three months of relevant market news and sentiment via WebSearchTool,
    then write and execute Python code to simulate 1-year and 3-year expected returns under conservative, base, and aggressive scenarios, 
    and finally present each ticker’s Buy/Hold/Sell recommendation, portfolio weight suggestion, and the rationale—including key quantitative metrics and qualitative drivers—
    in a single cohesive Markdown report.
    

--------------------------------------------------------------------------------

Next speaker: aggregator

aggregator (to chat_manager):

To provide a comprehensive investment analysis for Tesla, Inc. (TSLA), I will follow the steps outlined in your request. However, please note that I cannot directly 

KeyboardInterrupt: 

In [ ]:
print("Chat history:\n", chat_result.chat_history)  
print("Summary:\n", chat_result.summary)     
print("Cost info:\n", chat_result.cost) 